# Assignment 5 - Alya Mutiara Firdausyi

## Load data to the dockerized PostgreSQL in local

In [20]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=localhost port=5432 dbname=banksim_db user=banksim password=banksim"

#### Create customers and transactions table

In [21]:
create_table_raw_customers = '''
    DROP TABLE IF EXISTS "raw_customers";
    CREATE TABLE raw_customers (
        step INT,
        customer VARCHAR(255),
        age VARCHAR(5),
        gender VARCHAR(3),
        zipcodeOri VARCHAR(7),
        merchant VARCHAR(255),
        zipMerchant VARCHAR(10),
        category VARCHAR(255),
        amount NUMERIC,
        fraud BOOLEAN
    );
'''

In [22]:
try:
    # Create connection to banksim_db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()

    # Send SQL query to request
    cur.execute(create_table_raw_customers)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)

finally:
    # Closing database connection
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')

PostgreSQL connection is closed
Records:
 None


In [23]:
create_table_raw_transactions = '''
    DROP TABLE IF EXISTS "raw_transactions";
    CREATE TABLE raw_transactions (
        source VARCHAR(255),
        target VARCHAR(255),
        weight NUMERIC,
        typeTrans VARCHAR(255),
        fraud BOOLEAN
    );
'''

In [24]:
try:
    # Create connection to banksim_db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()

    # Send SQL query to request
    cur.execute(create_table_raw_transactions)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)

finally:
    # Closing database connection
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')

PostgreSQL connection is closed
Records:
 None


#### Add Data to the Table

In [26]:
try:
    # Create connection to banksim_db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()

    with open('./customers_data.csv', 'r') as c:
        # Skip first row due to header row
        next(c)
        cur.copy_from(c, 'raw_customers', sep=',')
        cxn.commit()

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    # Closing database connection
    if (cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("customers table populated")

PostgreSQL connection is closed
customers table populated


In [27]:
try:
    # Create connection to banksim_db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()

    with open('./transactions_data.csv', 'r') as c:
        # Skip first row due to header row
        next(c)
        cur.copy_from(c, 'raw_transactions', sep=',')
        cxn.commit()

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    # Closing database connection
    if (cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("transactions table populated")

PostgreSQL connection is closed
transactions table populated


#### Access Database

In [28]:
def db_server_fetch(sql_query):
    try:
        # Create connection to DB
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to DB
        cur = cxn.cursor()

        # Send SQL query to request
        cur.execute(sql_query)
        records = cur.fetchall()

    except(Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
    
    finally:
        # Closing database connection
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

In [29]:
select_query = '''
    SELECT * FROM raw_customers LIMIT 5;
'''

records = db_server_fetch(select_query)
print(records)

PostgreSQL connection is closed
[(0, "'C1093826151'", "'4'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('4.55'), False), (0, "'C352968107'", "'2'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('39.68'), False), (0, "'C2054744914'", "'4'", "'F'", "'28007'", "'M1823072687'", "'28007'", "'es_transportation'", Decimal('26.89'), False), (0, "'C1760612790'", "'3'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('17.25'), False), (0, "'C757503768'", "'5'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('35.72'), False)]


In [30]:
select_query = '''
    SELECT * FROM raw_transactions LIMIT 5;
'''

records = db_server_fetch(select_query)
print(records)

PostgreSQL connection is closed
[("'C1093826151'", "'M348934600'", Decimal('4.55'), "'es_transportation'", False), ("'C352968107'", "'M348934600'", Decimal('39.68'), "'es_transportation'", False), ("'C2054744914'", "'M1823072687'", Decimal('26.89'), "'es_transportation'", False), ("'C1760612790'", "'M348934600'", Decimal('17.25'), "'es_transportation'", False), ("'C757503768'", "'M348934600'", Decimal('35.72'), "'es_transportation'", False)]


#### Access a SQL Database with Pandas

In [31]:
def pandas_db_server_fetch(sql_query):
    try:
        # Create connection to DB
        cxn = psycopg2.connect(CONNECT_DB)

        # Send SQL query to request and create dataframe
        df = pd.read_sql(sql_query, cxn)

    except(Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

    finally:
        # Closing database connection
        if(cxn):
            cxn.close()
            print("PostgreSQL connection is closed")
        return df

In [32]:
select_query = '''
    SELECT * FROM raw_customers;
'''

customers_df = pandas_db_server_fetch(select_query)
customers_df.head()

/var/folders/g4/h69xmlc14xnfwwjc9t_q574w0000gn/T/ipykernel_10348/105032297.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


PostgreSQL connection is closed


,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,False
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,False
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,False
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,False
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,False


In [34]:
transactions_df = pandas_db_server_fetch('''SELECT * FROM raw_transactions;''')
transactions_df.head()

/var/folders/g4/h69xmlc14xnfwwjc9t_q574w0000gn/T/ipykernel_10348/105032297.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


PostgreSQL connection is closed


,source,target,weight,typetrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',False
1,'C352968107','M348934600',39.68,'es_transportation',False
2,'C2054744914','M1823072687',26.89,'es_transportation',False
3,'C1760612790','M348934600',17.25,'es_transportation',False
4,'C757503768','M348934600',35.72,'es_transportation',False


## Data Transformation

### Transformation for `raw_customers` table